---
title: "Signal EDA by Wine Category"
format: html
cdt: 2023-10-18T00:00:00
---


Back: [Thesis](src/wine_analysis_hplc_uv/notebooks/thesis.qmd) Subheading Profile Descriptions by Category


This document will contain a description of the signal profiles of each wine category by detection method. The intent is to provide an intuition into how models will behave when exposed to these categories, and see how similar they may or may not be. The hypothesis is that within each selected category there will be significant correlation, but also sufficient variance to uniquely identify each sample. The working hypothesis is also that DTW with a Sakoe-Chiba band of window size 10 will enable alignment without extraneous mutation.

We will start with the varietal category.


To perform this method, we need to connect all the separate pipes that resulted in the dataset used to test DTW. As per @bos_2020 [1679], the stages are:

1. denoise and smooth
2. baseline correction
3. retention time alignment
4. peak deonvolution and resolution enhancement
5. data compression

Now, our signals are sufficiently quiet and smooth to skip the first step, and we're not interested yet in stage 4, and stage 5. is achieved through resampling. So, we need:

1. get data
2. resampling
3. baseline correction

All relevant methods are in mindex_signal_processing SignalProcessor but no full pipeline method has been established yet. Lets build one with a single sample as the test subject.


In [ ]:
# set up environment

%reload_ext autoreload
%autoreload 2

import pandas as pd
from wine_analysis_hplc_uv import definitions
import seaborn as sns
import seaborn.objects as so
from wine_analysis_hplc_uv.old_signal_processing.signal_processor import (
    SignalProcessor,
)
import matplotlib.pyplot as plt
from scipy import signal
from wine_analysis_hplc_uv.notebooks import eda_by_category_methods

sns.set_theme(rc={"figure.dpi": 100})

plotter = eda_by_category_methods.Plotting()

scipro = SignalProcessor()

data = pd.read_parquet(definitions.RAW_PARQ_PATH)
data.head()

In [ ]:
data = data.pipe(scipro.propipe)
data.head()

In [ ]:
data.plot()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

In [ ]:
a = data.pipe(scipro.most_correlated, "blinesub")

In [ ]:
data = data.pipe(scipro.dynamic_time_warping, "blinesub")
data

In [ ]:
relplot_df = (
    data.loc[:, pd.IndexSlice[:, :, :, ["blinesub", "aligned"]]]
    .melt(ignore_index=False)
    .reset_index()
    .assign(
        winelabel=lambda df: df.role + "_" + df.samplecode + "_" + df.wine,
        mins=lambda df: df.mins.dt.total_seconds() / 60,
    )
    .set_index("mins")
    .sort_index()
    .loc[0.0:20.0, :]
)

rp = sns.relplot(
    data=relplot_df,
    x="mins",
    y="value",
    hue="subsignal",
    col="winelabel",
    col_wrap=2,
    kind="line",
    errorbar=None,
    height=3,
    aspect=2,
)
fig, ax = plt.subplots(1)

hm = plotter.alignment_heatmap(data, ax=ax, signal_label="blinesub")

Its not actually clear if modifying the window size is having an effect on the alignment.. need a mathematical description of the alignment. Lets start with peak detection.


## Peak Detection


In [ ]:
# get the peak indices for each sample, join the resulting series with the data and add
# a sparse column containing values where peaks are detected by boolean masking
peaks = (
    data
    # go to long form df for groupby operations
    .melt(ignore_index=False)
    # groupby 'samplecode' and select 'value' series
    .groupby("samplecode")["value"]
    # find peaks based on the given parameters, the peak index array is the zeroth
    # element of the returned tuple
    .apply(
        lambda value: pd.Series(
            signal.find_peaks(
                value,
            )[0]
        )
    )
    # return to a frame
    .to_frame(name="idx")
    # move 'samplecode' from index to column
    .reset_index("samplecode")
    # add a column 'ispeak' that contains True values to be used to identify the peak
    # elements after the join
    .assign(ispeak=True)
    # set the index as 'samplecode', 'idx' to prepare for join with the df
    .set_index(["samplecode", "idx"])
)
display(peaks)

How do I index the original frame with the found peaks to get the peak values, and preferably assign a boolean peak column alongside the signal column?

One method could be to massage the df into the same shape as the peak df, i.e. long, indices by group. once thats achieved a `where` call should enable me to mark the values corresponding to the indice and the samplecode


In [ ]:
(
    data
    # convert the timedelta index to float for easier plotting
    .pipe(lambda df: df.set_axis(axis=0, labels=df.index.total_seconds() / 60))
    # slice to the aligned signal from 0 to 16 minutes
    .loc[0:16, pd.IndexSlice[:, :, :, "aligned"]]
    # go long form for groupby operations
    .melt(ignore_index=False)
    # add a incremental index to match the find_peaks results
    .assign(idx=lambda df: df.groupby("samplecode").cumcount())
    # set the index as samplecode and the newly formed idx
    .reset_index()
    .set_index(["samplecode", "idx"])
    # left join based on samplecode and idx
    .join(peaks, on=["samplecode", "idx"])
    # any rows who did not have a corresponding peak element are NaN, now filled with False
    .assign(ispeak=lambda df: df.ispeak.fillna(False))
    # add a 'peak' column that is equal to the peak value using 'ispeak' as a mask on 'value'
    .assign(peak=lambda df: df["value"].loc[df.ispeak])
    # go to default index
    .reset_index()
    # add an overlay plot of the signals and their peaks
    .pipe(
        lambda df: so.Plot(df, x="mins", color="samplecode")
        .layout()
        .add(so.Line(), y="value")
        .add(so.Dot(), y="peak")
    )
)

from @scipy_findpeaks_2023: "a peak or local maximum is defined as any sample whose two direct neighbours have a smaller amplitude." It notes that noisy signals can result in errors due to loss of information about local maxima. In these cases they recommend using `find_peaks_cwt`, or exploring smoothing options.

Considering the results of above, perhaps it would be a good idea to experiment with smoothing.


In [ ]:
data

In [ ]:
# the above prototyped code has been wrapped in a class and placed in 'eda_by_category_methods'

processor = eda_by_category_methods.Processing()


def find_peaks_1(df, kwargs: None) -> pd.Series:
    a = signal.find_peaks(df.signal, **kwargs)[0]
    peaks = df.iloc[a].signal
    return peaks


data = data.loc[lambda df: df.signal_label == "blinecorr"].assign(
    peaks=lambda df: df.groupby("samplecode", group_keys=False).apply(find_peaks_1)
)


data.pipe(processor.find_peaks)

Based on the documentation of [chromatograpR](https://ethanbass.github.io/chromatographR/articles/chromatographR.html#pre-processing-data) we could use some smoothing, especially on 176, which is coincidentally our selected reference. But what is smoothing? Also, They use parametric time awrping or variable penalty dynamic time warping for alignment. They then use 'complete-linkage hierarchical clustering' to link peaks across samples.

Unfortunately there does not seem to be a Python library that implements penalty dynamic time warping, so lets focus on smoothing for now.

As per previous studies, first base is a Savitzky-Golay filter, which is implemented by SciPy. @cuadros-rodríguez_2021 used a 5 point window and second order polynomial. The SciPy implementation is a 1D filter that requires the data, a window length and polyorder. There are also a number of other parameters. It returns an array.


In [ ]:
sns.set(font_scale=1)

data = pd.read_parquet(definitions.RAW_PARQ_PATH)
f = processor.process(data)
display(f)

As we can see, no effect. The smoothing necessary to remove those detected peaks will result in unsatisfactory loss of signal information. Ergo better to use constraints in the peak detection algo. Also, remaining in simple long form with no multiindex massively reduces reshaping overhead and makes UDF functions much simpler to define..

Now lets add kwargs for peak finder.. added.


In [ ]:
display(data)
pro_data = processor.process(data, find_peak_kwargs=dict(height=6))

pro_data

In [ ]:
so.Plot(pro_data.loc[lambda df: df.mins < 21], x="mins", color="samplecode").add(
    so.Line(), y="signal"
).add(so.Dot(), y="peaks")

For testing purposes, we're only interested in the top 10 peaks


In [ ]:
# get top 10 peaks per sample
pro_data = pro_data.assign(
    select_peaks=lambda df: df.groupby("samplecode", group_keys=False)[
        "peaks"
    ].nlargest(10)
)

# plot the peaks on top of the curves
(
    pro_data.loc[lambda df: df.mins < 21]
    .pipe(
        lambda df: (
            df
            if so.Plot(df, x="mins", color="samplecode")
            .add(so.Line(), y="signal")
            .add(so.Dot(), y="select_peaks")
            else df
        )
    )
    # peak table
    .pipe(
        lambda df: (
            df
            if display(
                df.loc[:, ["samplecode", "wine", "mins", "select_peaks"]]
                .dropna()
                .assign(n_peak=lambda df: df.groupby("samplecode").cumcount())
                .pivot(
                    columns=["samplecode", "wine"],
                    index=["n_peak"],
                    values=["mins", "select_peaks"],
                )
                .reorder_levels([1, 2, 0], axis=1)
                .sort_index(axis=1)
            )
            else df
        )
    )
)

Now align, get the top 10 peaks again, and compare them.


In [ ]:
pro_data.head()

In [ ]:
# apply dtw

# find reference
def reference(df):
    reference = (
        df.corr().mean().loc[lambda df: df == df.max()]
        # .pipe(scipro.dynamic_time_warpingi)
    )
    return reference.index


reference = reference(
    pro_data.pivot_table(
        columns=["samplecode", "wine"], index=["mins"], values="signal"
    )
)

display(reference)

In [ ]:
# align

ref_signal = pro_data.loc[
    pro_data.samplecode == reference.get_level_values("samplecode")[0]
].signal.reset_index(drop=True)

(
    pro_data.set_index(["mins"])
    .groupby("samplecode", group_keys=False)["signal"]
    .apply(scipro.align_query_to_ref, ref_signal)
)

2023-10-18 Due to the fact that I am prototyping a lot of code and changing approaches (multiindex column default to long defaul primarily), literate progrmaming style is failing, re. version one of this notebook is somewhat unsalvagable - a testament to why rushing always fails in the end. At some point I'll have to go through and fix it. in the meantime, copy code over and push ON.


In [ ]:
%reload_ext autoreload
%autoreload 2

# set up environment
import pandas as pd
from wine_analysis_hplc_uv import definitions
import seaborn.objects as so
from wine_analysis_hplc_uv.old_signal_processing.signal_processor import (
    SignalProcessor,
)
from wine_analysis_hplc_uv.notes.data_analysis import eda_by_category_methods

plotter = eda_by_category_methods.Plotting()

scipro = SignalProcessor()
data = pd.read_parquet(definitions.RAW_PARQ_PATH)
data.head()

As we can see, no effect. The smoothing necessary to remove those detected peaks will result in unsatisfactory loss of signal information. Ergo better to use constraints in the peak detection algo. Also, remaining in simple long form with no multiindex massively reduces reshaping overhead and makes UDF functions much simpler to define..

Now lets add kwargs for peak finder.. added.


In [ ]:
# time std and drop vars level
dtwprocess = eda_by_category_methods.DTWProcessing()
sdata = dtwprocess.std_time(data)
sdata.head()

In [ ]:
# smooth

smdata = dtwprocess.smooth(sdata, "samplecode", "signal", "smooth_signal")
smdata.head()

In [ ]:
# add bline

bcdata = dtwprocess.blinecorr(
    smdata, "samplecode", "smooth_signal", bcorr_label="bcorr"
)

bcdata.head()

In [ ]:
# plot baseline correction

(
    bcdata.melt(
        id_vars=["samplecode", "wine", "mins"], var_name="siglabel", value_name="sig"
    ).pipe(
        lambda df: so.Plot(data=df, x="mins", y="sig", color="siglabel")
        .facet(col="samplecode", wrap=2)
        .add(mark=so.Line())
    )
)

In [ ]:
# detect peaks prior to alignment

peaks = eda_by_category_methods.Peaks()

pddata = peaks.peakdetect(
    bcdata, grouper="samplecode", signal_label="bcorr", col_label="unaligned_peaks"
)


pddata.head()

In [ ]:
# plot to display peaks
(
    pddata.melt(
        id_vars=["samplecode", "wine", "mins"], var_name="siglabel", value_name="sig"
    )
    .loc[lambda df: (df.mins < 5) & (df.mins > 3)]
    .pivot_table(index=["samplecode", "wine", "mins"], columns="siglabel", values="sig")
    .reset_index()
    .pipe(
        lambda df: so.Plot(data=df, x="mins", color="samplecode")
        .add(so.Line(), y="bcorr")
        .add(so.Dot(), y="unaligned_peaks")
    )
)

For testing purposes, we're only interested in the top 20 peaks


In [ ]:
# get top 20 peaks of the unaligned set

tpdata = peaks.top_peaks(
    pddata, "samplecode", "unaligned_peaks", 20, "top_20_unaligned"
)
tpdata

In [ ]:
# plot the peaks on top of the curves

peak_y = "top_20_unaligned"
display(
    tpdata.loc[lambda df: df.mins < 21].pipe(
        lambda df: (
            so.Plot(df, x="mins", color="samplecode")
            .add(so.Line(), y="bcorr")
            .add(so.Dot(), y=peak_y)
            .plot()
        )
    )
)

# peak table
()

In [ ]:
peaks.peak_table(tpdata, ["samplecode", "wine"], "top_20_unaligned", "peak_n", "mins")

Now align, get the top 10 peaks after alignment and compare them.


Notes: Use transform with a custom function if you are expecting a result with the same shape as the input i.e. a column of the dataframe. Apply rarely gets this right.


In [ ]:
# ref = ApplyDTW().find_ref(tdata, 'samplecode','bcorr')

dtwalign = eda_by_category_methods.ApplyDTW()

# aligned data
adata = dtwalign.align(
    tpdata, primary_key="samplecode", siglabel="bcorr", aligned_label="aligned"
)
adata.head()

In [ ]:
# plot aligned dataset

(
    adata.loc[lambda df: df.mins < 20].pipe(
        lambda df: so.Plot(data=df, x="mins", color="samplecode")
        .layout(size=(5, 3))
        .add(so.Line(), y="aligned")
    )
)

In [ ]:
# plot aligned against unaligned set
(
    adata.loc[lambda df: df.mins < 20]
    .melt(
        id_vars=["mins", "samplecode", "wine"],
        var_name="siglabel",
        value_vars=["bcorr", "aligned"],
        value_name="sig_val",
    )
    .pipe(
        lambda df: so.Plot(data=df, x="mins", color="siglabel")
        .layout(size=(15, 10))
        .add(so.Line(), y="sig_val")
        .facet(col="samplecode", wrap=2)
    )
)

In [ ]:
# detect peaks in aligned set
# aligned peaks data
apddata = peaks.peakdetect(
    adata, "samplecode", signal_label="aligned", col_label="peaks_aligned"
)

apddata.head()

In [ ]:
# add column of top 20 peaks in aligned peak set
# top aligned peaks data
tappdata = apddata.assign(
    aligned_top_20=lambda df: df.groupby("samplecode", group_keys=False)[
        "peaks_aligned"
    ].nlargest(20)
)
tappdata.info()

In [ ]:
# construct and display peak table

aligned_peak_table = tappdata.pipe(
    peaks.peak_table,
    grouper=["samplecode", "wine"],
    peaks_label="aligned_top_20",
    peak_num_label="peak_n",
    peak_idx_label="mins",
)
aligned_peak_table

In [ ]:
# plot overlay signals and top 20 peaks

(
    tappdata.loc[lambda df: df.mins < 20].pipe(
        lambda df: so.Plot(df, x="mins", color="samplecode")
        .add(so.Line(), y="aligned")
        .add(so.Dot(), y="aligned_top_20")
        .facet(col="samplecode", wrap=2)
        .layout(size=(15, 10))
    )
)

In [ ]:
# plot overlay aligned set, unaligned peaks, aligned peaks as dots

plot_peak_data = tappdata.melt(
    id_vars=["mins", "samplecode", "wine"],
    value_vars=["aligned_top_20", "top_20_unaligned"],
    var_name="peak_label",
    value_name="sig",
).dropna()

(
    tappdata.loc[lambda df: df.mins < 20].pipe(
        lambda df: so.Plot(df, x="mins")
        .add(so.Line(), y="aligned", color="samplecode")
        .add(
            so.Dot(),
            data=plot_peak_data.loc[lambda df: df.mins < 20],
            alpha="peak_label",
            color="samplecode",
            x="mins",
            y="sig",
        )
        # .facet(col='samplecode',wrap=2)
        .layout(size=(15, 10))
    )
)

As we can see in the above graphic, The majority of the peaks have lost amplitude, and torbreck-struie has straight up lost a peak at 18 mins. We need a better DTW algorithm.

Need to develop a more reliable alignment method. But we also should look at whether torbreck-struie itself is the issue. First what happens if we exclude it during warping?

Continue this in another book. export the processed data to parquet


In [ ]:
tappdata.loc[:, ["mins", "samplecode", "wine", "signal", "bcorr"]].to_parquet(
    "/Users/jonathan/mres_thesis/wine_analysis_hplc_uv/src/wine_analysis_hplc_uv/notebooks/processed_data.parquet"
)

# Section 3

2023-10-19

Looking at the effects of DTW over categories. Specifically, we're seeing an uncomfortable amount of warping in the most deviated samples, including significant losses of prominant peaks, which is an unacceptable mutation. Regardless of whether it is specific to that one sample (torbreck-struie) or not, we dont want to see any loss of prominant peaks, or peak mutation whatsoever. We are only expecting movement in the X, not the y.

Peak mutation is caused by masking with the warping path, where the data point with a given peaks maxima is excluded, resulting in a peak mutation, generally a decline in peak maxima. What it boils down to is a need for a warping algorithm that is punished for not aligning peak maxima. We could first experiment with the base DTW algorithm but the literature documentation @ethanbass_2023 shows that there are better algorithms out there, I just need to access them. Re: ChromatographR.
